# <center>Google Play and Apple Music App Purchases - Dec 2022 Version</center>

This project is an analysis of free apps obtained from Google Play and Apple.  The analysis answers the following question:

- Since we are developing a free app and will rely on advertising for income, which type of free app is most likely to attract users on Google Play and the Apple Store?

Import Pandas and a sample of free apps from the Apple Store and Google Play

In [1]:
import pandas as pd
df_apple = pd.read_csv('AppleStore.csv')
df_google = pd.read_csv('googleplaystore.csv')

Summary view of the imported data sets

In [2]:
a = "APPLE"
print(a.center(80))
print('\n', df_apple.head(3))
print('\n',"rows and columns", df_apple.shape)
print(2*'\n')
g = "GOOGLE"
print(g.center(80))
print('\n',df_google.head(3))
print('\n',"rows and columns", df_google.shape)

                                     APPLE                                      

           id      track_name  size_bytes currency  price  rating_count_tot  \
0  284882215        Facebook   389879808      USD    0.0           2974676   
1  389801252       Instagram   113954816      USD    0.0           2161558   
2  529479190  Clash of Clans   116476928      USD    0.0           2130805   

   rating_count_ver  user_rating  user_rating_ver      ver cont_rating  \
0               212          3.5              3.5     95.0          4+   
1              1289          4.5              4.0    10.23         12+   
2               579          4.5              4.5  9.24.12          9+   

         prime_genre  sup_devices.num  ipadSc_urls.num  lang.num  vpp_lic  
0  Social Networking               37                1        29        1  
1      Photo & Video               37                0        29        1  
2              Games               38                5        18        1  

 r

Based on review of the documentation for the Apple file (https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps) and the Google file (https://www.kaggle.com/datasets/lava18/google-play-store-apps) the following column headings may
be useful for the analysis:<br><br>
<b>APPLE</b><br>

- ratings_count_ver: Number of user ratings for current version of app
- user_rating_ver: Average user rating for current version of app
- prime_genre: Primary genre of the app

<b>GOOGLE</b><br>

- Category
- Reviews
- Installs
- Genres


### <center>DATA CLEANING</center>

In [3]:
#Per documentation, google row id 10472 is incorrect.  Check this row
df_google.loc[10472]

App               Life Made WI-Fi Touchscreen Photo Frame
Category                                              1.9
Rating                                               19.0
Reviews                                              3.0M
Size                                               1,000+
Installs                                             Free
Type                                                    0
Price                                            Everyone
Content Rating                                        NaN
Genres                                  February 11, 2018
Last Updated                                       1.0.19
Current Ver                                    4.0 and up
Android Ver                                           NaN
Name: 10472, dtype: object

In [4]:
#Per project instructions, delete row 10472. Comment out after executing to avoid deleting another row

df_google = df_google.drop(labels = 10472, axis=0)

#verified row index 10472 has been removed

Remove duplicate entries using the pandas code in the next two Notebook boxes.  Duplicates are determined
by duplicate "track_name" in the Apple set, and "App" in the Google set.  Kept the entry that contained the
most rating reviews, under the assumption that the highest review total is the most recent version of the app.

In [5]:
#remove duplicate rows Apple
df_apple = df_apple.sort_values(by= "rating_count_tot", ascending=False)
df_apple = df_apple.drop_duplicates(subset = "track_name",keep = "first")
print(df_apple.shape)

(7195, 16)


In [6]:
#remove duplicate rows Google
df_google = df_google.sort_values(by= "Reviews", ascending=False)
df_google = df_google.drop_duplicates(subset = "App",keep = "first")
print(df_google.shape)

(9659, 13)


Next we'll remove non English apps.  if a column "App"(Apple) or "track_name" (Google) has an entry
containing a 3 or more ASCII characters with character value > 127, we'll assume this entry is non English and remove.

In [7]:
df_apple = df_apple.drop(df_apple.loc[df_apple["track_name"]
           .apply(lambda x: False if len([i for i in x if ord(i) >127])<=3 else True)].index)   

In [8]:
df_google = df_google.drop(df_google.loc[df_google["App"]
            .apply(lambda x: False if len([i for i in x if ord(i) >127])<=3 else True)].index)   

Next isolate the free apps. First get the unique values and data types in the price column, then use that information to drop the non free apps.

In [9]:
#Apple prices and the column data type
print(df_apple["price"].unique())
print(df_apple["price"].dtypes)

[  0.     1.99   0.99   6.99   2.99   7.99   4.99   9.99   3.99   8.99
   5.99  14.99  13.99  19.99  17.99  15.99  24.99  20.99  29.99  12.99
  39.99  74.99  16.99 249.99  11.99  27.99  49.99  59.99  22.99  18.99
  99.99  21.99  34.99 299.99]
float64


In [10]:
#Google prices and the column data type
print(df_google["Price"].unique())
print(df_google["Price"].dtypes)

['0' '$2.49' '$2.99' '$0.99' '$3.99' '$4.99' '$399.99' '$1.99' '$79.99'
 '$7.49' '$4.49' '$9.99' '$5.99' '$13.99' '$4.29' '$1.70' '$14.99'
 '$12.99' '$1.49' '$19.99' '$1.50' '$3.49' '$1.29' '$7.99' '$29.99'
 '$1.59' '$11.99' '$89.99' '$39.99' '$3.02' '$6.99' '$2.90' '$1.00'
 '$19.40' '$5.49' '$5.00' '$1.75' '$8.99' '$3.08' '$18.99' '$299.99'
 '$17.99' '$379.99' '$8.49' '$2.56' '$15.46' '$2.95' '$1.97' '$19.90'
 '$24.99' '$2.60' '$4.80' '$1.26' '$4.77' '$400.00' '$16.99' '$10.99'
 '$4.59' '$389.99' '$33.99' '$46.99' '$1.76' '$3.90' '$2.00' '$2.50'
 '$10.00' '$14.00' '$3.95' '$6.49' '$3.28' '$1.20' '$2.59' '$4.84'
 '$37.99' '$1.61' '$4.60' '$9.00' '$15.99' '$3.88' '$74.99' '$3.04'
 '$4.85' '$154.99' '$1.04' '$25.99' '$1.96' '$28.99' '$109.99' '$200.00'
 '$30.99' '$3.61' '$394.99']
object


In [11]:
#Keep only the free Apple apps
df_apple = df_apple.drop(df_apple.loc[df_apple["price"]
            .apply(lambda x: False if x == 0 else True)].index)

In [12]:
#Keep only the free Google apps
df_google = df_google.drop(df_google.loc[df_google["Price"]
            .apply(lambda x: False if x == '0' else True)].index)

In [13]:
#check that only free Google apps left
print(df_google["Price"].unique())

['0']


In [14]:
#check that only free Apple apps left
print(df_apple["price"].unique())

[0.]


### <center>DATA ANALYSIS</center>

Determine the counts  and percentage of total for the types of apps in the datasets 

In [15]:
#Apple app type count and % of total
apple_count = pd.DataFrame(df_apple["prime_genre"].value_counts()).rename(columns = {"prime_genre": "App_count"})
apple_count["%_of_total"] = (apple_count/apple_count.sum()*100).round(decimals=1)
print(apple_count.head())

                   App_count  %_of_total
Games                   1872        58.1
Entertainment            254         7.9
Photo & Video            160         5.0
Education                118         3.7
Social Networking        106         3.3


In [16]:
#Google app type count and % of total
google_count = pd.DataFrame(df_google["Category"].value_counts()).rename(columns = {"Category": "App_count"})
google_count["%_of_total"] = (google_count/google_count.sum()*100).round(decimals=1)
print(google_count.head())

           App_count  %_of_total
FAMILY          1679        18.9
GAME             859         9.7
TOOLS            749         8.5
BUSINESS         407         4.6
LIFESTYLE        346         3.9


For Apple apps the Games genre dominates, with 58% of the total apps.  Next closest is Family with 8% of the total.
For Google, Family is the largest, with 19% of the total apps.  Games comes next with 9.7%.

This only gives the number of apps in the categories; we still need an indication of how popular the apps are.  Google provide number of installs, but only by category (100+, 10,000+, etc.)  Apple does not provide number of installs.  We'll use number of reviews as a proxy for number of installs.  For Google we'll use the numeric portion of the number of installs to estimate the value.

In [17]:
apple_count["total # of reviews"] = df_apple.groupby(by="prime_genre")["rating_count_tot"].sum()
print(apple_count.sort_values(by="total # of reviews", ascending = False).head(10))

                   App_count  %_of_total  total # of reviews
Games                   1872        58.1            42705795
Social Networking        106         3.3             7584125
Photo & Video            160         5.0             4550647
Music                     66         2.0             3783551
Entertainment            254         7.9             3563577
Shopping                  84         2.6             2261254
Sports                    69         2.1             1587614
Health & Fitness          65         2.0             1514371
Utilities                 81         2.5             1513441
Weather                   28         0.9             1463837


In [18]:
df_google["Installs"]= df_google["Installs"].str.replace("[+,]","").astype(int)
google_count["total # of installs"] = df_google.groupby(by="Category")["Installs"].sum()
pd.set_option('display.float_format', '{:.2E}'.format)
print((google_count.sort_values(by="total # of installs", ascending = False)).head(10))

                    App_count  %_of_total  total # of installs
GAME                      859    9.70E+00             1.34E+10
COMMUNICATION             287    3.20E+00             1.10E+10
TOOLS                     749    8.50E+00             8.00E+09
FAMILY                   1679    1.89E+01             6.20E+09
PRODUCTIVITY              345    3.90E+00             5.79E+09
SOCIAL                    236    2.70E+00             5.49E+09
PHOTOGRAPHY               261    2.90E+00             4.65E+09
VIDEO_PLAYERS             159    1.80E+00             3.93E+09
TRAVEL_AND_LOCAL          207    2.30E+00             2.89E+09
NEWS_AND_MAGAZINES        248    2.80E+00             2.37E+09


C:\Users\drrdm\AppData\Local\Temp\ipykernel_11864\2209650973.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_google["Installs"]= df_google["Installs"].str.replace("[+,]","").astype(int)


### <center>DISCUSSION & CONCLUSION</center>

Games dominates the app count and # of reviews for the Apple store.  GAME is the 2nd highest percentage of app total and has the highest total number of installs on Google. This is a popular but crowded market where it may be difficult to gain any market share.

Social networking has the 2nd highest number of reviews in the Apple store but is fourth in app count.  Social networking is popular but is much less crowded than Games.

COMMUNICATION is the second most installed app type with Google, ranking fifith in app count. SOCIAL ranks seventh in app count and ranks sixth in installs.

Based on this analysis, a social networking app which allows communication between individuals and groups may be our best opportunity.  Further analysis of demographics being served by current social networking apps may guide us to an underserved market, increasing the probability of success.